<a href="https://colab.research.google.com/github/daphnei/gen-ai-tutorials/blob/main/llm_tutorial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Imports and initialization
%pip install datasets
%pip install openai
%pip install cohere
%pip install textwrap 

from abc import ABC
import datasets
import openai
import cohere
from textwrap import wrap
from IPython.display import clear_output 

COHERE_SECRET_KEY = None
OPENAI_SECRET_KEY = None

class Engine(ABC):
  def score(self, text: str) -> tuple[list[str], list[float]]:
    """Tokenizes and scores a piece of text.
    
    The score is log-likelihood. A higher score means a token was more
    likely according to the model.
    
    Returns a list of tokens and a list of scores.
    """
    pass

  def generate(self, prompt, top_p, num_tokens):
    """Generates text given the provided prompt text."""
    pass


# Glossary

Here are some of the terms we will be using in this tutorial.


*   **(Language) Model**: A neural network trained to generate text.
*   **Engine**: The code and computers used to do inference with a model.
*   **Accelerator**: A [GPU](https://www.techtarget.com/searchvirtualdesktop/definition/GPU-graphics-processing-unit) or [TPU](https://cloud.google.com/tpu/docs/intro-to-tpu) hardware chip used to massively speed up model inference by enabling fast, parallelized matrix multiplcations. 



# Unit 1: Working with Pre-Trained Languages Models

## 1.1 Choosing a model and engine

Which model should you use? It depends on what your goals are, what your budget is, and what kinds of computational resources you have available.

In this section, we will summarize the pros and cons some of the popular systems, and guide you through the process of setting them up for inference.

## Open-source models on HuggingFace

HuggingFace is an open-source code framework for loading an open-source models onto an accelerator in order to train or do inference. It is very popular among academic researchers.

There are [hundreds of models](https://huggingface.co/models) available through HuggingFace's model repository. Some prominent ones which you can use for tutorial are.

### Why use this?
HuggingFace is great if your goal is to write open-source code, with results that anyone can reproduce. HuggingFace gives access to a huge number of models, and it is fairly easy to swap between models.

Because HuggingFace is widely used, it is easy to find help online. Probably someone else has had the same question as you and already posted about it.

### Why not use this?
When you use HuggingFace, you are running the code on your own computer. If you don't have a big enough accelerator, you will run into difficulties loading up larger models. For example, Colab (the software you are currently using) gives you access to a Tesla T4 GPU, which has 16 GB of RAM. This means, you can load the 6.7B parameter LLaMa, but not any of the bigger ones.

* **[Pythia-3B](https://huggingface.co/EleutherAI/pythia-2.8b-deduped)**: Part of a family of models trained by [Eleuther AI](https://www.eleuther.ai/about), a non-profit AI research lab.
* **[GPT-2 XL](https://huggingface.co/gpt2-xl)**: The original large language model from OpenAI, and the last one they open-sourced before moving away from open source models.
* **[BLOOM-3b](https://huggingface.co/docs/transformers/model_doc/bloom)**: These models were created by the [BigScience Initative](https://bigscience.huggingface.co/), a collaboration between HugginFace and many academic research labs to responsibly build a set of high-quality multilingual models.

## Open-source LLaMA 

Released this past February by Meta Research, LLaMA is the latest-and-greatest in open-source pre-trained LLMs. The LLaMA models range in size from 7 billion to 65 billion paramters.

Since the models' release, they have been finetuned by researchers to improve functionality for a variety of use cases. Some prominent derivatives of LLaMA are:

* **Alpaca**: Alpaca is introduced by researchers from Stanford. It is finetuned for academic research and to promote accessibility to models with capabilities similar to closed-source models like OpenAI's text-davinci-003. Based on Meta's LLaMA 7B model, Alpaca is specifically trained on 52k instruction-following demonstrations in the style of self-instruct using text-davinci-003. It has demonstrated similar behaviors to OpenAI's text-davinci-003 in various aspects while being smaller and more easily reproducible. The training recipe and data of Alpaca are open-sourced, and the model weights will also be released in the future.

* **Vicuna**: Vicuna is introduced through a collaboration between researchers from UC Berkeley, CMU, Stanford, and UC San Diego. It is finetuned for addressing the limitations of existing large language models (LLMs) like OpenAI's ChatGPT. This open-source chatbot is fine-tuned from the LLaMA base model and trained on approximately 70K user-shared ChatGPT conversations sourced from ShareGPT.com using public APIs. A preliminary evaluation with GPT-4 as the judge demonstrated that Vicuna-13B achieves over 90% quality compared to OpenAI ChatGPT and Google Bard, while surpassing models like LLaMA and Stanford Alpaca in over 90% of cases.

* **TODO**:
 
While LLaMa and its derivative models can be used with the HuggingFace framework, the Colab runtime you are on right now doesn't have a big enough accelerator to load any of these models.

Instead, we have loaded up LLaMA on LTI's compute cluster, and you can use the code below to query the cluster. Note that the server being queries in the code below will be taken down at the end of the tutorial.

### Why use this?

LLaMA is a state-of-the-art foundational large language model designed to help researchers advance their work in this subfield of AI.  

* Accessibility: LLaMA is more efficient and less resource-intensive than other models, and it is available under a non-commercial license to researchers and other organizations. LLaMA is available in various sizes(7B, 13B, 33B, and 65B parameters), making it accessible to a range of computing resources. 

* Open-source Community: LLaMA models are part of the open-source ecosystem, users can benefit from the extensive community support, documentation, and shared resources available through platforms like HuggingFace.

### Why not use this?
Unlike OpenAI's GPT models, which run computations on OpenAI's servers, LLaMA models require substantial computational resources to load and run. Access to a sufficiently sized accelerator or compute cluster is necessary to handle the computational demands. Additionally, efforts are needed for infrastructure setup, serving framework implementation, and API development.

For this tutorial, the LLaMA models are hosted on the LTI compute cluster, and instructions to access the models will be provided below.

### Setup Instructions
TODO

In [ ]:
#run this cell to query the llama model hosted on cmu server
import llm_client

#update the address and port 
client = llm_client.Client(address="cluster-address", port="5000")

class LlamaEngine(Engine):
  def __init__(self):
    pass

  def score(self, text):
    pass

  def generate(self, prompt, top_p, num_tokens):
    response = client.prompt(
      prompt,
      top_p=top_p,
      temperature=1.0,
      max_new_tokens=num_tokens,
      repetition_penalty=1.0
    )
    return response[0].text

engine = LlamaEngine()

In [ ]:
print(engine.generate("I visited Pittsburgh to", 0.5, 10))

## Cohere's models

### Setup Instructions

1. Create an account [here](https://dashboard.cohere.ai/welcome/register).
2. Go to the [API key page](https://dashboard.cohere.ai/api-keys), and copy the `TRIAL` API key, then paste it into the code block below.
3. The `TRIAL` API Key allows you to make 5 API calls per-minute. If you would like to make more calls than that, go to the [API key page](https://dashboard.cohere.ai/api-keys) and press the "Get your production key" button. You will need to agree to their terms of service and provide credit card details in order to receieve a `PRODUCTION` API key.
4. Copy your API key, and then run the code block below. It will ask you to enter your secret key into a text box.

In [ ]:
#@title Run this code block to use Cohere for Unit 1.

import cohere

if COHERE_SECRET_KEY is None:
  print("Please paste your API key here:")
  COHERE_SECRET_KEY = input().strip()
co = cohere.Client(COHERE_SECRET_KEY)
clear_output()

class CohereEngine(Engine):
  def __init__(self):
    pass

  def score(self, text):
    response = co.generate(
      prompt=text,
      num_generations=1,
      max_tokens=0,
      return_likelihoods="ALL"
    )
    tokens = [t.token for t in response[0].token_likelihoods]
    likelihoods = [t.likelihood for t in response[0].token_likelihoods]
    return tokens, likelihoods

  def generate(self, prompt, top_p, num_tokens):
    response = co.generate(
      prompt=prompt,
      num_generations=1,
      p=top_p,
      temperature=1.0,
      max_tokens=num_tokens,
      frequency_penalty=0.0,
      presence_penalty=0.0,
      return_likelihoods="ALL"
    )
    return response[0].text

engine = CohereEngine()

In [ ]:
print(engine.generate("I visited Pittsburgh to", 0.5, 10))
print(engine.score("I visited Pittsburgh to see the CMU campus."))
print(engine.score("I visited Pittsburgh to see flying hippos."))

## OpenAI's GPT-3/4

OpenAI's GPT-3 (Generative Pre-trained Transformer 3) and GPT-4 are highly advanced language models developed by OpenAI. These models have gained significant attention due to their impressive capabilities in generating human-like text and performing various natural language processing tasks.  

#### GPT-3:
GPT-3 is the third iteration in the GPT series and represents a significant advancement in language modeling. With a whopping 175 billion parameters, GPT-3 is currently one of the largest language models ever created. It has been trained on a vast amount of diverse internet text, allowing it to generate coherent and contextually relevant responses to prompts. GPT-3's size enables it to exhibit impressive performance across a wide range of language tasks, such as text completion, language translation, summarization, question answering, and more. It has shown promising results in creative writing, conversational agents, and even programming assistance.

#### GPT-4:
As the newest version of OpenAI's Large Language Model, it is a multimodal large language model. GPT-4 is capable of accepting both text and image inputs and output human-like text. GPT-4 has made significant improvement on GPT-3 in aspects such as handling longer prompts, performing tasks on basic mathematics and processing programming instructions, etc.

### Why use these?
OpenAI's model have become the industry standard for large language models.
They have an API which is very easy to use. Since the models all get run on OpenAI's servers, you don't need your own compute resources, and there is practically no setup involved to get started.

### Why not use these?
OpenAI's models are not open-source and only accessible through an API. There is relatively little information available on how they were trained or what data they were trained on. OpenAI may change the models being used under the hood by the API, and you as a user will not know it. This makes them a bad choice for fully reproducible research.

The OpenAI API costs money to use. See [this link](https://openai.com/pricing) for pricing details.

### Setup Instructions
1. Go to www.openai.com and create an account.
2. Go to https://platform.openai.com/account/api-keys and click the "Create new secret key" button. It doesn't matter what you name it.
3. Create a file `secrets/openai-api-key`, and paste your key in the file.

In [65]:
#@title Run this code block to use GPT-3 for Unit 1.
MODEL_NAME = "text-davinci-003" #@param ["gpt-4", "text-davinci-003", "text-curie-001", "text-babbage-001", "text-ada-001"]

import openai

if OPENAI_SECRET_KEY is None:
  print("Please paste your API key here:")
  OPENAI_SECRET_KEY = input().strip()
openai.api_key = OPENAI_SECRET_KEY
clear_output()

class OpenAIEngine(Engine):
  def __init__(self, model_name):
    self.model_name = model_name

  def score(self, text):
    response = openai.Completion.create(
        engine=self.model_name,
        prompt=text,
        max_tokens=0,
        logprobs=1,
        echo=True)

    tokens = response["choices"][0]["logprobs"]["tokens"]
    logprobs = response["choices"][0]["logprobs"]["token_logprobs"]
    if logprobs and logprobs[0] is None:
      # GPT-3 API does not return logprob of the first token
      logprobs[0] = 0.0
    return tokens, logprobs
    
  def generate(self, prompt, top_p=1.0, num_tokens=32, num_samples=1):
    response = openai.Completion.create(
      engine=self.model_name,
      prompt=prompt,
      temperature=1.0,
      max_tokens=num_tokens,
      top_p=top_p,
      frequency_penalty=0.0,
      n=num_samples,
      presence_penalty=0.0,
      logprobs=1
    )
    outputs = [r["text"] for r in response["choices"]]
    return outputs[0] if num_samples == 1 else outputs

engine = OpenAIEngine(MODEL_NAME)

## 1.3 Experimenting with Generation

Let's check that the engine you chose actually works as expected. Try generating some text by running the following code block. You can control the following parameters:

- Set `num_tokens` to control the length of the generation.
- Set 'top_p` to control the amount of randomness. (You'll learn more about this in Unit 1.4.)
- Set `num_samples' to control how many generations the engine outputs. Avoid setting this too high or things might get really slow.

In [71]:
prompt = "The hippopotamus is"
samples = engine.generate(prompt, top_p=1.0, num_tokens=32, num_samples=2)

for idx, sample in enumerate(samples):
  print(f"\nSAMPLE {idx}")
  print(sample)


SAMPLE 0
 a large semi-aquatic mammal native to sub-Saharan Africa. It is the third largest land mammal after the elephant and rhinoceros. Hippos

SAMPLE 1
 an African mammal famous for its bulky size and semi-aquatic lifestyle. It is the third largest land mammal after elephants and rhinoceroses, reaching


### ⭐⭐⭐ YOUR TURN ⭐⭐⭐

Try your own prompt in the code above.

## 1.3 Analyzing Likelihoods

As we discussed in the lecture, language models take as input a prompt sequence and then output a score (log-probability) for each token in the vocabulary. A higher score means the model is more confident this that this token fits as the next token in the sequence.

In this unit, we will inspect token likelihoods to build an understaning of why models give tokens higher or low scores.

To put this into context, let's consider the sequence "It's raining outside". We should expect the token "outside" to receive a high score since it's a plausible continuation to the prefix "It's raining".
However, if we replace "outside" with "oustide" (typo), or with "inside" (semantically nonsensical), we should expect the score to be substantially lower.
Let's try this out!

In [3]:
s = "It's raining outside"
s_typo = "It's raining oustide"
s_nonsense = "It's raining inside"

s_tokens, s_scores = engine.score(s)
s_typo_tokens, s_typo_scores = engine.score(s_typo)
s_nonsense_tokens, s_nonsense_scores = engine.score(s_nonsense)

print("Sequence:", s)
for token, score in zip(s_tokens, s_scores):
    print(f"[{token}]: {score:.2f}")

print("Sequence:", s_typo)
for token, score in zip(s_typo_tokens, s_typo_scores):
    print(f"[{token}]: {score:.2f}")

print("Sequence:", s_nonsense)
for token, score in zip(s_nonsense_tokens, s_nonsense_scores):
    print(f"[{token}]: {score:.2f}")


Sequence: It's raining outside
[It]: 0.00
['s]: -1.78
[ raining]: -6.21
[ outside]: -3.03
Sequence: It's raining oustide
[It]: 0.00
['s]: -1.78
[ raining]: -6.21
[ oust]: -15.12
[ide]: -0.27
Sequence: It's raining inside
[It]: 0.00
['s]: -1.78
[ raining]: -6.21
[ inside]: -8.21


The perplexity of a language model on a sequence of words captures how "surprise" it is by this sequence. Unlikely sequences, such as "It's raining inside" should lead to high perplexity.

Perplexity is used as the standard metric for how good a language model is at modeling human language. For example GPT-3 Davinci has a perplexity of 20.5 on the Penn Treebank corpus, a remarkable improvement from its predecessor GPT2-1.5B (35.76).

Now let's implement the perplexity function. Recall that perplexity is defined as `2 ^ -normalized_sequence_logprob`, where `normalized_sequence_logprob` is the sum of scores of all tokens in the sequence, divided by the length of the sequence.

In [ ]:

def perplexity(log_probs: list[float]) -> float:
    assert log_probs, "cannot compute perplexity on an empty sequence!"
    normalized_sequence_logprob = sum(log_probs) / len(log_probs)
    return 2 ** -normalized_sequence_logprob

s_ppl = perplexity(s_scores)
s_typo_ppl = perplexity(s_typo_scores)
s_nonsense_ppl = perplexity(s_nonsense_scores)

print(f"Sequence: {s}\nPerplexity: {s_ppl:.2f}\n")
print(f"Sequence: {s_typo}\nPerplexity: {s_typo_ppl:.2f}\n")
print(f"Sequence: {s_nonsense}\nPerplexity: {s_nonsense_ppl:.2f}\n")

Sometimes it can unintuitive what text has high- or low- perplexity according to a language model.

Consider the following two poems stanzas:

**Poem A**
```
’Twas brillig, and the slithy toves
      Did gyre and gimble in the wabe:
All mimsy were the borogoves,
      And the mome raths outgrabe.
```

**Poem B**
```
’Twas crillig, and the brithy lokes
      Did ryne and jimble in the waze:
All timsy were the habogroves,
      And the nome paths intrabe.
```
These seem equally nonsense, right?

Run the code block below to observe their perplexities. One of these poems is much more likely than the other!

This is because Poem A is an excerpt of "[Jabberwocky](https://www.poetryfoundation.org/poems/42916/jabberwocky)," a famous poem by the author Lewis Carrol. It is very likely to have appeared in the language model's training dataset several times, so the model assigns it a very low perplexity, even though it is gibberish.

In contrast, Poem B was written by your tutorial instructor and most definitely is not in any model training set (yet).

In [ ]:
poem_a = """’Twas brillig, and the slithy toves
      Did gyre and gimble in the wabe:
All mimsy were the borogoves,
      And the mome raths outgrabe.
"""
_, poem_a_scores = engine.score(poem_a)

poem_b = """’Twas crillig, and the brithy lokes
      Did ryne and jimble in the waze:
All timsy were the habogroves,
      And the nome paths intrabe.
"""
_, poem_b_scores = engine.score(poem_b)

print(f"Poem A\nPerplexity: {perplexity(poem_a_scores):.2f}\n")
print(f"Poem B\nPerplexity: {perplexity(poem_b_scores):.2f}\n")

### ⭐⭐⭐ YOUR TURN ⭐⭐⭐

Copy and paste a sentence from a news article, book, Tweet, or elsewhere. Observe the sentence's perplexity. Can you make the sentence higher or lower perplexity by swapping out words or phrases with different ones?

## 1.4 Controlling the Amount of Randomness During Generation

In this section, we will investigate the impact the `top_p` parameter has on the text that a model generates.

As we went over in the lecture, valid values for $p$ range from 0 to 1. 
Higher values of $p$ mean more randomness in the generation (becasue, the model can choose from more possible vocabulary items). Lowever values of $p$ restrict the number of vocabulary items the generation algorithm can chose between.

In [31]:
# When p is set to 1.0, we get many different answers.
prompt = "Once upon a time, there was a dragon named"
p = 1.0
generations = engine.generate(prompt, top_p=p, num_tokens=2, num_samples=5)
print(f"GENERATIONS WITH P={p}")
for gen in generations:
  print(gen)

# When p is set to a lower value there is much less diversity.
p = 0.2
generations = engine.generate(prompt, top_p=p, num_tokens=2, num_samples=5)
print(f"\nGENERATIONS WITH P={p}")
for gen in generations:
  print(gen)

# When p is set to 0, all the generations are exactly the same.
p = 0.0
generations = engine.generate(prompt, top_p=p, num_tokens=2, num_samples=5)
print(f"\nGENERATIONS WITH P={p}")
for gen in generations:
  print(gen)

GENERATIONS WITH P=1.0
 Blueberry
 Drake.
 Finn.
 Gugar
 Narus

GENERATIONS WITH P=0.2
 Drake.
 Draco.
 Draco.
 Kail
 Kair

GENERATIONS WITH P=0.0
 Bob.
 Bob.
 Bob.
 Bob.
 Bob.


### ⭐⭐⭐ YOUR TURN ⭐⭐⭐

Let's try generating a coupple hundred words. You can either use the prompt provided below or pick your own prompt.

You should notice that the model seems more creative when $p$ is set higher, but it is also more prone to weird word choices.

In [37]:
import textwrap

In [45]:
### ⭐⭐⭐ Experiment yourself ⭐⭐⭐

prompt = "Once upon a time, there was a fairy who lived in a datacenter."

p = 1.0
print(f"\nGENERATION WITH P={p}")
generations = engine.generate(prompt, top_p=p, num_tokens=100, num_samples=1)
print(textwrap.fill(generations[0]))

p = 0.0
print(f"\nGENERATION WITH P={p}")
generations = engine.generate(prompt, top_p=p, num_tokens=100, num_samples=1)
print(textwrap.fill(generations[0]))


GENERATION WITH P=1.0
 She spent her days helping humans maintain their servers, granting
them more processing power when the workload was too high, and
protecting their webservers from malicious attacks.  One day, the
fairy was approached by a powerful wizard. The wizard asked her for
her help in creating a magical portal to another dimension. He
promised her great rewards if she completed the task, so the fairy
agreed to help.  The wizard gave the fairy an ancient scroll with
instructions on how to build the portal. She

GENERATION WITH P=0.0
 She was a very powerful fairy, and she had the ability to control all
of the computers in the datacenter.  One day, the fairy was feeling
bored and decided to play a game. She created a program that would
randomly select a computer in the datacenter and then ask the user a
question. If the user answered the question correctly, the fairy would
reward them with a magical item.  The fairy was delighted with her
game and soon many of the users in 

## 1.5 Creating a Classifer using In-Context Learning

### What is a classifier?
Language models can be used for generation, but they are also commonly used to build classification systems. These are systems which take as input some text and assign a label to it. Some examples tasks:

- Classify Tweets as TOXIC or NOT_TOXIC
- Classify restaurant reviwers as having POSITIVE or NEGATIVE sentiment.
- Classify two statements as AGREEing with each other or CONTRADICT-ing each other.

In this section, you will learn how to build a binary sentiment classifier using an LLM using a technique called few-shot learning.

### What is In-Context learning?

Getting a pre-trained language model to do a particular task you want it to do can be challenging. Language models know a lot of things, but without tuning or conditioning, they often struggle to understand the format of the task you are asking of them.

For this reason, it is common to write prompts which tell the language model the format of the task they are supposed to be accomplishing. This approach is called **in-context learning.** When the provided prompt contains several examples of the task the model is supposed to accomplish, this is called **few-shot learning**, since the model is learning to do the task from a few examples.

Writing the perfect prompt is more an art than a science, but a prompt often has
two parts:
- An `instruction` string that instructs the model on how to complete the task.
This is especially import for models like OpenAI's `text-davinci-003` which
are fine-tuned to follow user instructions.
- Several `demonstration` strings (the "shots" in few-shot) that give examples of completing the task.

For example, suppose we want to design a prompt for the task of translating words from English to Chinese. A prompt for this task could look like
the following:

```
Translate English to Chinese.

dog -> 狗
apple -> 苹果
coffee -> 咖啡
supermarket -> 超市
squirrel ->
```

Given this prompt, most large language models should answer with the correct answer: "松鼠."

Now, let's build a binary sentiment classifier for [Yelp reviews](https://huggingface.co/datasets/yelp_polarity/).

In [ ]:

yelp = datasets.load_dataset("yelp_polarity")
train_data = yelp["train"].shuffle(seed=1).select(range(4))
test_data = yelp["test"].shuffle(seed=1).select(range(20))

print("Data format:", train_data[1])


In [ ]:
# convert integer labels to text
label_map = {
    0: "negative",
    1: "positive"
}

instruction = "Classify the sentiment of these yelp reviews as positive or negative.\n\n"
prompt_template = "Review: {review}\nSentiment: {sentiment}\n\n"

# construct the prompt by concatenating instructions and templates
prompt_parts = [instruction]
for instance in train_data:
    review = instance["text"]
    sentiment = label_map[instance["label"]]
    prompt_parts.append(prompt_template.format(review=review, sentiment=sentiment))
prompt = ''.join(prompt_parts)


print("YOUR PROMPT:", prompt, sep='\n')

In [ ]:
# now we can feed the prompt to GPT-3 to classify a new review!

eval_template = "Review: {review}\nSentiment: {sentiment}"

def classify_review(review: str) -> str:
  """ Classify a single movie review """
  label_to_score = {}
  for label in label_map.values():
    label_prompt = prompt + eval_template.format(review=review, sentiment=label)
    _, score = engine.score(label_prompt)
    label_score = score[-1]
    label_to_score[label] = label_score
  
  return max(label_to_score, key=label_to_score.get)


def evaluate(verbose: bool=False) -> float:
  """ Evaluate your prompt on the test set """
  correct = []
  for i, instance in enumerate(test_data):
    review = instance["text"]
    label = label_map[instance["label"]]
    predicted = classify_review(review)
    correct.append(1 if label == predicted else 0)

    if verbose:
      print(f"======== {i+1} / {len(test_data)} ========")
      print(f"REVIEW: {review}")
      print(f"LABEL: {label}")
      print(f"PREDICTED: {predicted}")

  acc = sum(correct) / len(correct)
  return acc


acc = evaluate(verbose=True) # should get 100% performance
print(f"Accuracy of your prompt on {len(test_data)} test examples: {acc:.0%}")

## 1.6 Creating an Explanation Generator using Few-Shot Learning

We can also use few-shot learning techniques to guide generation. Let's build a system that identifes puns.

In [58]:
instruction = "Explain each pun.\n\n"
few_shot_examples = [
    ('I made a pun about the wind but it blows.', '"Blows" can either refer to moving air, or it refer to the pun being not very good.'),
    ("What washes up on tiny beaches? Microwaves.", "A microwave is either a kitchen appliance or it could mean a tiny wave."),
    ("I'm no cheetah, you're lion!", 'The feline "lion" sounds like the word "lying," while the feline "cheetah" sounds like "cheating."')]
prompt_template = "Pun: {pun}\nExplanation: {explanation}\n\n"

prompt_parts = [instruction]
for pun, explanation in few_shot_examples:
    prompt_parts.append(prompt_template.format(pun=pun, explanation=explanation))
prompt = ''.join(prompt_parts)
print(prompt)

Explain each pun.

Pun: I made a pun about the wind but it blows.
Explanation: "Blows" can either refer to moving air, or it refer to the pun being not very good.

Pun: What washes up on tiny beaches? Microwaves.
Explanation: A microwave is either a kitchen appliance or it could mean a tiny wave.

Pun: I'm no cheetah, you're lion!
Explanation: The feline "lion" sounds like the word "lying," while the feline "cheetah" sounds like "cheating."




In [63]:
query_pun = "Long fairy tales have a tendency to dragon."
prompt_with_query = prompt + "Pun: {pun}\nExplanation:".format(pun=query_pun)
print(engine.generate(prompt_with_query, top_p=0.0, num_tokens=32))

 "Dragon" can refer to a mythical creature, or it can mean to drag on.


### ⭐⭐⭐ YOUR TURN ⭐⭐⭐

Experiment with changing up the order of the few-shot examples above, or deleting then and writing your own. Can you get the languague model to do this task using only an instruction and no few-shot examples? Can you come up with examples that work better than the provided ones?

# Unit 2: Models Tuned for Instruction-Following and Dialog

## 2.1 Choosing a model

## 2.2 Comparing Behaviour of Pre-trained and Tuned Models

## 2.3 Building a Persona Bot with In-Context Learning

# Scratch
This is where Daphne is keeping random code pieces which will not end up in the tutorial.

In [ ]:
engine.generate("What can", top_p=1.0, num_tokens=4)

In [ ]:
response = openai.Completion.create(
      engine="text-davinci-003",
      prompt="hey what's going here",
      temperature=1.0,
      max_tokens=24,
      logprobs=1,
      echo=True
    )

In [ ]:
offset = response["choices"][0]["logprobs"]["text_offset"]
logprobs = response["choices"][0]["logprobs"]["token_logprobs"]
text = response["choices"][0]["text"]

In [ ]:
for left, right, logprob in zip(offset, offset[1:] + [len(text)], logprobs):
    print("token:", text[left:right])
    print("logprob", logprob)